In [40]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

- เราจะใช้ข้อมูลหลายขนาด เพื่อแสดงให้เห็นว่าการทำ Nested Cross-Validation นั้นสามารถปรับค่า hyperparameters และประเมินโมเดลได้ดีกว่าการทำ Cross-Validation ธรรมดา

In [48]:
# Load Iris data
iris = load_iris()
X = iris.data  # ใช้ทั้งหมดของ features
Z = iris.data[:, :1]  # ใช้เฉพาะ feature แรกเพื่อให้เหมาะกับ Polynomial Regression แต่ผลคือ R2 ติดลบ 
y = iris.target

X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [42]:
# ฟังก์ชันสำหรับคำนวณความเที่ยงตรง (Precision) และความแม่นยำ (Accuracy)
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return rmse, r2

In [43]:
# Cross Validation
def cross_validation(X, y, k=10, degrees=[1, 2, 3, 4]):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    rmse_values = []
    r2_values = []
    best_degrees = []
    
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = Pipeline([
            ('poly', PolynomialFeatures()),
            ('linear', LinearRegression())
        ])
        param_grid = {'poly__degree': degrees}
        
        grid_search = GridSearchCV(model, param_grid, cv=KFold(n_splits=inner_k, shuffle=True, random_state=42), scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        best_degree = grid_search.best_params_['poly__degree']
        
        y_pred = best_model.predict(X_test)
        rmse, r2 = evaluate_model(y_test, y_pred)
        
        rmse_values.append(rmse)
        r2_values.append(r2)
        best_degrees.append(best_degree)
        
        print(f"Cross-Validation Fold {fold + 1} - RMSE: {rmse:.4f}, R2: {r2:.4f}, Degree: {best_degree}")
    
    return np.mean(rmse_values), np.mean(r2_values), best_degrees

### ใน Nested Cross-Validation จะมีการแบ่งข้อมูลออกเป็นสองลูป:

1. ลูปภายนอก (Outer loop): ใช้สำหรับการประเมินโมเดล
2. ลูปภายใน (Inner loop): ใช้สำหรับการปรับค่า hyperparameters ของโมเดล

In [44]:
# Nested Cross Validation
def nested_cross_validation(X, y, outer_k=5, inner_k=3, degrees=[1, 2, 3, 4]):
    outer_kf = KFold(n_splits=outer_k, shuffle=True, random_state=42)
    outer_rmse_values = []
    outer_r2_values = []
    best_degrees = []

    for train_index, test_index in outer_kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = Pipeline([
            ('poly', PolynomialFeatures()),
            ('linear', LinearRegression())
        ])
        param_grid = {'poly__degree': degrees}
        
        inner_kf = KFold(n_splits=inner_k, shuffle=True, random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=inner_kf, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        best_degree = grid_search.best_params_['poly__degree']
        
        y_pred = best_model.predict(X_test)
        rmse, r2 = evaluate_model(y_test, y_pred)
        
        outer_rmse_values.append(rmse)
        outer_r2_values.append(r2)
        best_degrees.append(best_degree)
    
    return np.mean(outer_rmse_values), np.mean(outer_r2_values), best_degrees


- ค่า R2 ที่ใกล้เคียงกับ 1 มากที่สุดแสดงว่าโมเดลสามารถอธิบายความแปรปรวนของข้อมูลได้ดี ค่า R2 ที่ต่ำหรือเป็นลบแสดงว่าโมเดลไม่สามารถอธิบายความแปรปรวนได้ดี

- ค่า RMSE ที่ต่ำกว่าหมายถึงโมเดลที่มีความแม่นยำมากกว่า

In [45]:
# Run nested cross-validation
outer_k = 5
inner_k = 3

rmse_nested, r2_nested, best_degrees_nested = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k)
print(f"Nested Cross-Validation - RMSE: {rmse_nested:.4f}, R2: {r2_nested:.4f}, Best Degrees: {best_degrees_nested}")

# Run cross-validation
rmse_cv, r2_cv, best_degrees_cv = cross_validation(X, y, k=10, degrees=[1, 2, 3, 4])
print(f"Cross-Validation - RMSE: {rmse_cv:.4f}, R2: {r2_cv:.4f}, Best Degrees: {best_degrees_cv}")

Nested Cross-Validation - RMSE: 0.2275, R2: 0.9194, Best Degrees: [2, 1, 2, 1, 2]
Cross-Validation Fold 1 - RMSE: 0.2395, R2: 0.8976, Degree: 2
Cross-Validation Fold 2 - RMSE: 0.1742, R2: 0.9584, Degree: 2
Cross-Validation Fold 3 - RMSE: 0.1072, R2: 0.9777, Degree: 2
Cross-Validation Fold 4 - RMSE: 0.1896, R2: 0.9397, Degree: 2
Cross-Validation Fold 5 - RMSE: 0.2080, R2: 0.9406, Degree: 2
Cross-Validation Fold 6 - RMSE: 0.2457, R2: 0.8922, Degree: 2
Cross-Validation Fold 7 - RMSE: 0.2473, R2: 0.9161, Degree: 2
Cross-Validation Fold 8 - RMSE: 0.2101, R2: 0.9212, Degree: 2
Cross-Validation Fold 9 - RMSE: 0.1770, R2: 0.9530, Degree: 2
Cross-Validation Fold 10 - RMSE: 0.2503, R2: 0.8719, Degree: 1
Cross-Validation - RMSE: 0.2049, R2: 0.9268, Best Degrees: [2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
